In [33]:
#if rating is 4 or 5 then is a good review, if 1, 2 or 3 then its a abd review
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import re
import math

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
import pickle

In [35]:
data = pd.read_csv("Data.csv")
data.head(10)

,Unnamed: 0,overall,verified,reviewText,summary,Binary_Overall
0,0,5.0,True,This is the best novel I have read in 2 or 3 y...,A star is born,1.0
1,1,3.0,True,"Pages and pages of introspection, in the style...",A stream of consciousness novel,0.0
2,2,5.0,False,This is the kind of novel to read when you hav...,I'm a huge fan of the author and this one did ...,1.0
3,3,5.0,False,What gorgeous language! What an incredible wri...,The most beautiful book I have ever read!,1.0
4,4,3.0,True,I was taken in by reviews that compared this b...,A dissenting view--In part.,0.0
5,5,4.0,True,I read this probably 50 years ago in my youth ...,Above average mystery,1.0
6,6,5.0,True,I read every Perry mason book voraciously. Fin...,Lam is cool!,1.0
7,7,5.0,True,I love this series of Bertha and Lamb.. Great...,Five Stars,1.0
8,8,5.0,True,Great read!,Five Stars,1.0
9,9,4.0,False,"Crows Can't Count, A.A. Fair\n\nMr. Harry Shar...",A Fast and Far Moving Adventure,1.0


In [36]:
#sample beacuse daatset too big for my PC
data = data.sample(100000)
data

,Unnamed: 0,overall,verified,reviewText,summary,Binary_Overall
3415780,3416459,5.0,True,This is a great case for the Hisense Sero 7 Li...,Best case you can buy.,1.0
3010012,3010583,5.0,True,"Fast shipping, fits like a glove. Works great.",fits like a glove,1.0
4241079,4242043,5.0,True,I've been running my laptop with an old batter...,excellent laptop battery purchase!,1.0
2000940,2001293,5.0,True,These maybe small but the sound from these are...,Maxwell Ear buds,1.0
5786228,5787872,2.0,True,I purchased this as a audio solution for conso...,this adapter is not a good solution for this,0.0
...,...,...,...,...,...,...
220975,221019,5.0,True,I've used this device for 5 years now and have...,Great Device for Connecting Old TVs and Recorders,1.0
6334402,6336249,5.0,True,Great bundle,AS EXPECTED GREAT PRODUCT,1.0
3318288,3318943,5.0,True,Great,Five Stars,1.0
5113291,5114599,5.0,True,Great protection for my JBL player.,Five Stars,1.0


In [37]:
#prepare data for model. Balance classes. 
data.Binary_Overall.value_counts()

1.0    80997
0.0    19003
Name: Binary_Overall, dtype: int64

In [38]:
import nltk

In [39]:
X = data.reviewText
Y = data.Binary_Overall

In [40]:
X = X.replace(r"\n", ' ', regex = True)

In [41]:
xtrain,xtest,y_train,y_test=train_test_split(X,Y); 
print('train size:',len(xtrain))
print('test size:',len(xtest))

train size: 75000
test size: 25000


In [42]:
from imblearn.under_sampling import RandomUnderSampler

undersampler=RandomUnderSampler(sampling_strategy='majority');

X_train_us,y_train_us=undersampler.fit_resample(xtrain.values.reshape(-1, 1),y_train);

print('Composición del training set:')
print(y_train_us.value_counts())

print('\nComposición del test set:')
print(y_test.value_counts())

Composición del training set:
1.0    14254
0.0    14254
Name: Binary_Overall, dtype: int64

Composición del test set:
1.0    20251
0.0     4749
Name: Binary_Overall, dtype: int64


In [43]:
from nltk.corpus import stopwords 
stop_words=stopwords.words('english');

In [44]:
#Bayes model. Few parameters for both vectorizer and bayes to make prediction faster and not burn out my PC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB

vectorizer=TfidfVectorizer(stop_words=stop_words,strip_accents='unicode')

model=Pipeline([('vect',vectorizer),('classifier',MultinomialNB())])

params={'classifier__alpha':[0.1,0.5,1],'vect__max_df':[1,100],'vect__ngram_range':[(1,2),(2,2),(1,1)]}

GS_CV=GridSearchCV(model,params,cv=5,n_jobs=-1)

GS_CV.fit(X_train_us.ravel(), y_train_us);

print('best score:',GS_CV.best_score_)
print('best params:',GS_CV.best_params_)

best score: 0.7560689476410417
best params: {'classifier__alpha': 0.5, 'vect__max_df': 100, 'vect__ngram_range': (1, 2)}


In [49]:
#score is better than the multiclass classifier. Also f1 score

from sklearn.metrics import accuracy_score
pred = GS_CV.predict(xtest.values.astype("U"))
accuracy_score(y_test,pred)

0.69252

In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[ 4033,   716],
       [ 6971, 13280]], dtype=int64)

In [55]:
#testing self-made comments 
texto_prueba=['I think its cool']
k = GS_CV.predict(texto_prueba)
print('Comentario:',texto_prueba,'\n')
print("overall:" , k)
print(' proba:',GS_CV.predict_proba(texto_prueba).max())

Comentario: ['I think its cool'] 

overall: [1.]
 proba: 0.5407689131322051


In [56]:
texto_prueba=['Could be better']
k = GS_CV.predict(texto_prueba)
print('Comentario:',texto_prueba,'\n')
print("overall:" , k)
print(' proba:',GS_CV.predict_proba(texto_prueba).max())

Comentario: ['Could be better'] 

overall: [0.]
 proba: 0.5626191979596529


In [70]:
texto_prueba=['awsome']
k = GS_CV.predict(texto_prueba)
print('Comentario:',texto_prueba,'\n')
print("overall:" , k)
print(' proba:',GS_CV.predict_proba(texto_prueba).max())

Comentario: ['awsome'] 

overall: [1.]
 proba: 0.6501012937505533


In [71]:
import pickle
with open('model_binary.pkl', 'wb') as f_model:
    pickle.dump(GS_CV, f_model)